In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from xml import etree

## .gov Data

In [2]:
URL = "https://s3.eu-west-2.amazonaws.com/data.nationalhighways.co.uk/ha-roadworks/nh_roadworks_2022_6_27.xml"

response = requests.get(URL)
with open('2022_6_7.xml', 'wb') as file:
    file.write(response.content)

In [3]:
tree = ET.parse('2022_6_7.xml')
root = tree.getroot()

In [4]:
root = root[0][0]

In [5]:
children = []
for child in root:
    children.append(child.attrib)

In [6]:
df = pd.DataFrame.from_records(children)

In [7]:
df.columns

Index(['NEW_EVENT_NUMBER', 'SDATE', 'EDATE', 'EXPDEL', 'DESCRIPTION',
       'CLOSURE_TYPE', 'STATUS', 'PUBLISHED_DATE', 'OLD_REFERENCE_NUMBER'],
      dtype='object')

In [8]:
df['SDATE'] = pd.to_datetime(df['SDATE'])
df['EDATE'] = pd.to_datetime(df['EDATE'])
df['PUBLISHED_DATE'] = pd.to_datetime(df['PUBLISHED_DATE'])

In [9]:
df

,NEW_EVENT_NUMBER,SDATE,EDATE,EXPDEL,DESCRIPTION,CLOSURE_TYPE,STATUS,PUBLISHED_DATE,OLD_REFERENCE_NUMBER
0,00236665-001,2022-07-07 22:00:00,2022-07-08 05:00:00,Slight (less than 10 mins),A40 Westbound Jct 1\r\nSlip road lane closures...,Programmed Routine Works,Published,2021-10-08 16:42:51,NaN
1,00233760-003,2022-07-11 20:00:00,2022-07-23 06:00:00,Moderate (10 - 30 mins),A1(M)/M62 Link roads (A1(M) Jct 41/M62 Jct 32a...,Programmed Routine Works,Published,2021-10-21 11:51:46,NaN
2,00031362-003,2020-09-01 00:00:00,2023-12-01 00:00:00,Slight (less than 10 mins),A30 Carland Cross to Chiverton Cross improveme...,Major Schemes,Published,2021-10-15 08:02:48,NaN
3,00242874-001,2022-07-08 15:00:00,2022-07-08 21:00:00,Slight (less than 10 mins),Women's European Football Championships\r\nSpa...,Off Network,Published,2021-11-18 16:13:19,4231242
4,00113644-005,2021-12-08 00:00:00,2024-07-01 06:00:00,Slight (less than 10 mins),A5 from Bucks Head Farm Bridge to Packington M...,Major Schemes,Published,2021-12-06 08:36:05,NaN
...,...,...,...,...,...,...,...,...,...
1319,00275330-001,2022-07-08 20:00:00,2022-07-09 05:00:00,Slight (less than 10 mins),A38 northbound Palm Court to Kedleston.\r\nLan...,Emergency Routine Works,Published,2022-06-24 12:11:32,NaN
1320,00275024-001,2022-06-28 20:00:00,2022-06-29 05:00:00,Slight (less than 10 mins),A47 westbound \r\nCastor to Sutton Roundabout ...,Programmed Routine Works,Published,2022-06-23 09:07:50,NaN
1321,00273279-002,2022-06-28 22:00:00,2022-06-29 05:00:00,Slight (less than 10 mins),A3 eastbound to Esher roundabot\r\nExit slip a...,Programmed Routine Works,Published,2022-06-21 09:59:37,NaN
1322,00274440-001,2022-06-26 21:00:00,2022-06-27 06:00:00,Moderate (10 - 30 mins),M6 Southbound Junction 9 to junction 8 leading...,Emergency Routine Works,Published,2022-06-21 11:19:27,NaN


In [ ]:
df.to_excel('2022_6_27.xlsx', index=False)

## TFL Data

In [11]:
import requests, json

In [12]:
def get_json_data(url) -> list:
    try:
        hdr ={
        # Request headers
        'Cache-Control': 'no-cache',
        }

        response = requests.get(url, headers=hdr)
    except Exception as e:
        print(e)
        return False
    return response.json()

def convert_json_to_df(json: list) -> pd.DataFrame:
    return pd.DataFrame.from_records(json)

def get_df_from_endpoint(url: str) -> pd.DataFrame | bool:
    json = get_json_data(url)
    if json:
        return convert_json_to_df(json)
    return False

In [13]:
# All roads managed by tfl.

roads_url = "https://api.tfl.gov.uk/Road/"
all_roads_df = get_df_from_endpoint(roads_url)

In [21]:
# Disruptions by Date
disruptions_url = "https://api.tfl.gov.uk/Road/all/Street/Disruption?startDate=01-01-2022&endDate=01-01-2023"
disruptions_df = get_df_from_endpoint(disruptions_url)

In [22]:
disruptions_df.to_excel('Exploring disruptions.xlsx')